# Volume 2, Chapter 14: RAG Fundamentals

**Build a simple RAG system for network documentation**

From: AI for Networking Engineers - Volume 2, Chapter 14

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eduardd76/AI_for_networking_and_security_engineers/blob/main/CODE/Colab-Notebooks/Vol2_Ch14_RAG_Basics.ipynb)

## 🔧 Setup

In [ ]:
# Install required packages
!pip install -q langchain langchain-anthropic langchain-openai chromadb

In [ ]:
# Set API keys
import os
from getpass import getpass

if 'ANTHROPIC_API_KEY' not in os.environ:
    os.environ['ANTHROPIC_API_KEY'] = getpass('Enter Anthropic API key: ')

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass('Enter OpenAI API key (for embeddings): ')

print("✓ API keys set!")

## 📖 RAG Demo

### Step 1: Create Sample Network Documentation

In [ ]:
# Sample network documentation
network_docs = [
    """
    VLAN Configuration Standards
    
    VLAN Ranges:
    - VLAN 1-99: Infrastructure
    - VLAN 100-199: User networks
    - VLAN 200-299: Voice
    - VLAN 999: Quarantine
    
    Best Practices:
    - Never use VLAN 1 for production traffic
    - Document all VLANs in IPAM
    - Use meaningful VLAN names
    """,
    
    """
    BGP Configuration Standards
    
    AS Numbers:
    - Primary AS: 65001
    - Backup AS: 65002
    
    Peering Requirements:
    - MD5 authentication required
    - Prefix-list filtering mandatory
    - Maximum prefix limits: 1000 for peers
    - BFD for fast convergence
    """,
    
    """
    OSPF Design Guidelines
    
    Area Design:
    - Area 0: Backbone (core switches/routers)
    - Area 1: Data centers
    - Area 2: Branch offices
    
    Best Practices:
    - Limit areas to 50-100 routers
    - Use stub areas at network edges
    - Reference bandwidth: 100000 (for 100G links)
    """
]

print(f"✓ Created {len(network_docs)} sample documents")

### Step 2: Build RAG System

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

print("Step 1: Creating embeddings...")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

print("Step 2: Splitting documents into chunks...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = []
for doc in network_docs:
    chunks.extend(text_splitter.split_text(doc))

print(f"  → Split into {len(chunks)} chunks")

print("Step 3: Creating vector database...")
vectorstore = Chroma.from_texts(
    chunks,
    embeddings,
    persist_directory="./colab_chroma_db"
)

print("Step 4: Setting up LLM...")
llm = ChatAnthropic(
    model="claude-sonnet-4-20250514",
    temperature=0
)

print("Step 5: Creating RAG chain...")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

print("\n✓ RAG system ready!")

### Step 3: Ask Questions!

In [ ]:
# Try different questions
questions = [
    "What VLAN should I use for voice traffic?",
    "What is our primary BGP AS number?",
    "Which OSPF area should data centers be in?"
]

for i, question in enumerate(questions, 1):
    print(f"\n{'='*60}")
    print(f"Question {i}: {question}")
    print('='*60)
    
    result = qa_chain({"query": question})
    
    print(f"\nAnswer:\n{result['result']}")
    
    print(f"\nSources used:")
    for j, doc in enumerate(result['source_documents'], 1):
        print(f"  {j}. {doc.page_content[:80]}...")

### Try Your Own Question

In [ ]:
# Ask your own question
my_question = input("Ask a question about network standards: ")

result = qa_chain({"query": my_question})

print(f"\nAnswer:\n{result['result']}")

## 🎯 How RAG Works

1. **Index** - Documents are split into chunks and converted to embeddings
2. **Store** - Embeddings are stored in a vector database (ChromaDB)
3. **Retrieve** - When you ask a question, it finds relevant chunks
4. **Generate** - Claude uses the retrieved chunks to answer your question

## 💡 Key Benefits

- Works with your own documentation
- More accurate than asking Claude directly
- Shows sources (traceability)
- Can handle large knowledge bases

## 📚 Next Steps

- Add your own network documentation
- Try other RAG chapters (advanced retrieval, semantic search)
- Build a production RAG system